In [7]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import gc

gc.enable()

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 8,
    'min_data_in_leaf': 3300,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.97,
    'bagging_freq': 1,
    'lambda_l1': 0,
    'verbose': 0
}

#params = {
#    'task': 'train',
#    'boosting_type': 'rf',
#    'objective': 'binary',
#    'metric': {'auc'},
#    'num_leaves': 550,
#    'min_data_in_leaf': 6,
#    'feature_fraction': 0.55,
#    'bagging_fraction': 0.8,
#    'bagging_freq': 1,
#    'lambda_l1': 0,
#    'min_gain_to_split': 0,
#    'verbose': 0
#}

/home/ike/.miniconda3/envs/intel/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-09-29 07:47:55.171327. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [2]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/latest/"

In [18]:
# level1 param

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 2,
    'min_data_in_leaf': 400,
    'feature_fraction': 1,
    'bagging_fraction': 0.91,
    'bagging_freq': 1,
    'verbose': 0
}

In [3]:
def training(train, valid):
    X = lgb.Dataset(train.drop(['user_id', 'work_id', 'rating'], axis=1), train['rating'])
    V = lgb.Dataset(valid.drop(['user_id', 'work_id', 'rating'], axis=1), valid['rating'], reference=X)
    gbdt = lgb.train(params, X, valid_sets=[X,V], num_boost_round=200, early_stopping_rounds=20, verbose_eval=False)
    Yvp = gbdt.predict(valid.drop(['user_id', 'work_id', 'rating'], axis=1), num_iteration=gbdt.best_iteration)
    Ytp = gbdt.predict(train.drop(['user_id', 'work_id', 'rating'], axis=1), num_iteration=gbdt.best_iteration)
    return (roc_auc_score(train['rating'].values, Ytp), roc_auc_score(valid['rating'].values, Yvp))

In [4]:
def cv(param, paramlst):
    trainauc = [0.0]*len(paramlst)
    validauc = [0.0]*len(paramlst)
    for i, p in enumerate(paramlst):
        params[param]=p
        tv = [0,0,0]
        vv = [0,0,0]
        for fold in [1,2,3]:
            t = pd.read_csv(data+'train_{0}.csv'.format(str(fold)))
            v = pd.read_csv(data+'valid_{0}.csv'.format(str(fold)))
            #t['item_category'] = t.item_category.astype('category')
            #v['item_category'] = v.item_category.astype('category')
            tv[fold-1], vv[fold-1] = training(t, v)
        trainauc[i]=np.mean(tv)
        validauc[i]=np.mean(vv)
    paramtable = pd.DataFrame({
        'TrainingSet': trainauc,
        'ValidationSet': validauc
    }, columns=['TrainingSet', 'ValidationSet'], index=pd.Index(paramlst, name=param))
    print(paramtable)

In [8]:
cv('num_leaves', [4, 8, 20, 40, 100])

            TrainingSet  ValidationSet
num_leaves                            
4              0.905893       0.756573
8              0.905893       0.756573
20             0.905893       0.756573
40             0.905893       0.756573
100            0.905893       0.756573


/home/ike/.miniconda3/envs/intel/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-09-29 07:48:05.908150. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


LibFM

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_svmlight_file

path = "/mnt/d/Data/mangaki-data-challenge/libfm/"

In [25]:
params = {
    'n_iter':340000, 
    'init_stdev':0.01,
    'l2_reg_w':0.00025,
    'l2_reg_V':0.5,
    'rank':1,
    'step_size':0.02
}

In [13]:
from scipy.sparse import hstack
from fastFM import sgd

def train_fm(train, valid, trainy, validy):
    fm = sgd.FMClassification(**params)
    fm.fit(train, np.require(trainy*2-1, dtype=np.int))
    return (roc_auc_score(trainy, fm.predict_proba(train)), roc_auc_score(validy, fm.predict_proba(valid)))

def cv_fm(param, paramlst):
    trainauc = [0.0]*len(paramlst)
    validauc = [0.0]*len(paramlst)
    for i, p in enumerate(paramlst):
        params[param]=p
        tv = [0,0,0]
        vv = [0,0,0]
        for fold in [1,2,3]:
            train, ty = load_svmlight_file(path+"train_{0}.csv".format(fold))
            valid, vy = load_svmlight_file(path+"valid_{0}.csv".format(fold))
            ta = pd.read_csv(data+'train_{0}.csv'.format(str(fold))).fillna(0)
            va = pd.read_csv(data+'valid_{0}.csv'.format(str(fold))).fillna(0)
            tv[fold-1], vv[fold-1] = train_fm(train, valid, ty, vy)
        trainauc[i]=np.mean(tv)
        validauc[i]=np.mean(vv)
    paramtable = pd.DataFrame({
        'TrainingSet': trainauc,
        'ValidationSet': validauc
    }, columns=['TrainingSet', 'ValidationSet'], index=pd.Index(paramlst, name=param))
    print(paramtable)

In [26]:
cv_fm('l2_reg_V', [0.4, 0.5, 0.6])

          TrainingSet  ValidationSet
l2_reg_V                            
0.4          0.920071       0.806509
0.5          0.920067       0.806510
0.6          0.920065       0.806509
